# Vortex Lattice Method: Effect of Aspect Ratio and Wing Shape on Lift

*By Jenna Osborn*

*MAE 6226 Spring 2017*

For this project, my goal was to develop a model that allows someone to quickly assess the wing shape and aspect ratio on the coefficient of lift and drag for a particular style of wing. The only inputs required are...
1. Aspect Ratio
2. Number of panels in the chord direction
3. Number of panels in the span direction
4. Sweep back angle
5. Taper Ratio
6. Freestream velocity
7. Angle of Attack 

## Background

The study of finite wings is critical for the application of potential flow for real-life scenarios. The panel methods we have been using in class have been dealing with an inifinte 2D wing. 

A method that can be used to assess the lift on a 3D wing shape is the **Vortex Lattice Method** (VLM). The VLM panels the top surface of the wing into square or tetrahedrals. Once the top surface of the wing is panelled. A "bound" votex-sheet is placed 1/4 chord of the panel from the leading edge. Then trailing vortices are added to each end of the bound vortex sheet to give the effect of the trailing vortices. This creates a horse shoe vortex effect for a single panel. The circulation strength is assumed to be constant over the horseshoe vortex. 

Much like the 2D panel method, the circulation strength of the vortex sheet can be solved by a linear equation with the normal velocity components adding up to equal zero. The model below assumes a planar wing (no cambering) and that the wing is thin enough to ignore the the curvature to the cross-sectional airfoil, thus making it a 2D problem. 

### History of VLM 

Ludwig Prandtl and his collegues developed the first practical theory for a model for a finite wing. His model, the Lifting-Line theory is still used for early calculations for finite wing characteristics. His theory involves a single vortex sheet that runs along the span of the wing with two "Free-trailing" vortex filaments at the ends of the vortex sheet like a giant horseshoe vortex. This can be expanded into an infinte number of these horseshoe vortices along the span of the wing so that there can be a varying strength circulation along the span as seen in Figure 1. 

**ADD picture from Fundamentals of Aerodynamics pg 364**

In [4]:
Image(filename = )

SyntaxError: invalid syntax (<ipython-input-4-e0f3131257f9>, line 1)

This Lifting Line theory is a pretty good representation of straight wings, but falls short for low aspect ratio wings, swept wings, and Delta wings. On these shapes a more complicated model is required for a better representation for the aerodynamic characteristics. One of these more complicated models is the VLM. 

### The mathematics behind VLM

The model follows the mathematics of the model outline in *Aerodynamics for Engineers* by Bertin and Smith. For a more in depth outline of the mathematics, the reader is referred to this book. 

Thie model approximates the the continuous distribution of bound vorticity of the wing a a finite number of horseshoe vortices. The geometry is broken up into panels on which these horseshoe vortices are placed. The bound vortex part of the horseshoe vortex is placed at 1/4 chord on the panel and aligned with the panel's local sweepback angle. The ends of the horseshoe vortices are aligned parallel to the y axis. 

The normal velocity is required to be zero by this model and can be used to solve for the individual panel circulation strength. This boundary condition is applied to a control point on the panel which is located at the center of the panel in the x direction and 3/4 chord in the y direction as seen in Figure 2. 

The angle of attack of the freestream velocity is the with the y axis as seen in Figure 3. 

The model begins with the velocity induced by a vortex sheet of strength $\Gamma$ as given by the *Biot-Savart Law* 

$$dV = \frac{\Gamma(dl \times r)}{4\pi r^3}$$

For brevety purposes, following the model by Bertin and Smith the normal velocity induced on panel *m* by the horseshoe vortex on panel *n* is given by

$$ w_{m,n} = \frac{\Gamma_{n}}{4\pi} \left\{ \frac{1}{(x_m-x_{1n})(y_m-y_{2n}) -(x_m -x_{2n})(y_m-y_{1n})} \left[ \frac{(x_{2n}-x_{1n})(x_m-x_{1n})+(y_{2n}-y_{1n})(y_m - y_{1n})}{\sqrt{(x_m-x_{1n})^2+(y_m-y_{1n})^2}} - \frac{(x_{2n}-x_{1n})(x_m-x_{2n})-(y_{2n}-y_{1n})(y_m-y_{1n})}{\sqrt{(x_m-x_{1n})^2+(y_m-y_{2n})^2}} \right] +\frac{1.0}{y_{1n}-y_m} \left[ 1.0 + \frac{x_m-x_{1n}}{\sqrt{(x_m-x_{1n})^2 + (y_m-y_{1n})^2}} \right] - \frac{1.0}{y_{2n}-y_m} \left[ 1.0 + \frac{x_m-x_{2n}}{\sqrt{(x_m-x_{2n})^2 +(y_m-y_{2n})^2}} \right] \right\}  $$

In this equation $(x_m, y_m)$ are the control point coordinates on panel *m*, $(x_{1n}, y_{1n})$ is the left endpoint on of the vortex sheet on panel *n*, and $(x_{2n}, y_{2n})$ is the right endpoint on the vortex sheet on panel *n*.

The linear equation that must be solved is the sum of all the vortex contributions on the control point at panel *m* and the freestream velocity contribution 

$$ w_{m} = \displaystyle\sum_{n=1}^{2N} w_{m,n} $$



### VLM vs. Panel Methods



## Development of the Model

The model below is comprimised of small functions that are used to create the larger model called **VLM**. The model than can be used with the imputs as stated above for an assessment of the lift and drag.

#### Importing the Necessary Libraries 

In [3]:
import numpy
from scipy import linalg 
from matplotlib import pyplot
from IPython.core.display import Image

#embed the figures inline with the text
%matplotlib inline

#### Geometry and Associated Panels 

The first function needed for completion of the model is to take the inputted values of Aspect Ratio (AR), number of panels in the chord direction (nchord), number of panels in the spanwise direction (nspan), sweep angle of the leading edge of the wing (sweepangle), the taper ratio of the wing if a tapered wing (tapratio), and Type of wing and then to return a set of points for the panels on the wing.

This model assumes symmetry about the middle of the wing, so only one of the sides needs to be discretized into panels and calculated. The points can be reflected about the y axis for the other half. The resulting lift can be doubled at the end. We are going to define the starboard half of the wing and we will deal with the port side half later. 

The different Types of wings are dealt with separately. 

1. **DELTA WING**
    - The delta wing is assumed to be a triangle wing. The panels are rectilinearly defined. Due to the shape of the triangle, each row will have a different number of panels. So, only the nchord variable is necessary to define. The nspan, sweepangle, tapratio can all be left as 0. 
    
    **?Picture here?**
    
2. **CONSTANT CHORD**
    - The  constant chord model is assumed to have a constant chord throughout the span of the wing. This chord is calculated from the inputted Aspect ratio. The nchord, nspan, and sweepangle all can be varied as an input. The tapratio is not necessary in this model because the ratio of the inside edge to the outide edge is always 1. This input can be left as 0.

3. **TAPERED**
    - The tapered model allows the input to vary the ratio of the wing to the outside egde chord length to the inside edge chord length. This is a more realistic version of different set of wings. The nspan, nchord, sweepangle, and tapratio can be set by the input to varying values. 

The next set of functions to define is calculating the bound vortex sheet end points and the control points for each panel. The triangle panels have to be handled differently than the square or trapezoidal panels. The bound vortex endpoints are calculated from the 1/4 chord on one side of the triangle point on the other side. The control point is still in the middle of the panel and 3/4 chord from the top of the panel as if it were a square. 

